In [1]:
pip install tensorflow==2.15.0


  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.13.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-4.25.7-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.14.1-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl.m

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\jaina\\AppData\\Local\\Temp\\pip-unpack-73y1_pnw\\tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl'
Consider using the `--user` option or check the permissions.



In [1]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

import pickle
import pandas as pd
import numpy as np


In [ ]:
#

In [2]:
##load all the ANN trained model ,scaler pickle file ,one hot encoding
model=load_model('model.h5')

##load the encoderans scaler

with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)
    
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)
    
    
    

In [3]:
#exmaple INput data
input_data={
    
    'CreditScore': 600,
    'Geography' : 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary': 50000
}


In [4]:
#
# one hot encode

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()

# Use correct column names
geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Users\jaina\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
#encode 
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
 #combine 1 hot encode with inputs data
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
#Scaling rhe data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [9]:
##prediction churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


array([[0.07275809]], dtype=float32)

In [10]:

prediction_probe=prediction[0][0]

In [11]:
prediction_probe

np.float32(0.072758086)

In [12]:
if prediction_probe >0.5:
    print(" The customer is likely to churn")
else:
    print('the customer is not likely to churn')

the customer is not likely to churn
